In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time

from tqdm import tqdm
from urllib.parse import urljoin

In [2]:
import tbwriters_utils
import pandas as pd

## Here we will extract all the articles links from https://www.tbwriters.com/ webpage


Tabs not extracted:



------------
-----------
------------
------------
------------

In [35]:
def loop_article_page(custom_url, key_code):
    """
    
    """
    return_file = {
        "Data": [],
        "message": "success",
        "response": 200
    }
    All_url_links = {}
    
    print(custom_url, key_code)
    try:
        for i in tqdm(range(1, 1000)):
            if i == 1:
                final_url = custom_url
            else:
                second_part = "&paged=" + str(i)
                final_url = custom_url + second_part
            
            if custom_url == "http://www.tbwriters.com":
                second_part = "/?paged=" + str(i)
                final_url = custom_url + second_part
            
            found_url_links, load_more = tbwriters_utils.extract_all_tbwriters_article_links(final_url)
            key = key_code + str(i)
            All_url_links[key] = found_url_links
            print(final_url, load_more)
            if found_url_links["Response"] != 200:
                print(found_url_links["Message"], found_url_links["Response"])
            
            if load_more == False or found_url_links["Response"] == 404:
                break
                
        return_file["Data"] = All_url_links
        return return_file
    
    except Exception as e:
        print(e)
        return_file["Data"] = All_url_links
        return_file["message"] = e
        return_file["response"] = 404
        return return_file

In [36]:
def check_error_in_links(All_url_link, page_code, print_each_error=False):
    """
    
    
    """

    error_counter = 0
    for page_id in range(1, len(All_url_link)):
        page_key = page_code + str(page_id)
        try:
            All_url_link.get(page_key)
            if  All_url_link.get(page_key)["Response"]!= 200:
                error_counter += 1
                if print_each_error:
                    print(page_key, All_url_link.get(page_key)["message"])
        except Exception as e:
            print(page_key, e)

    print(f"Total error in {page_code}: {error_counter}")

In [37]:
def save_json(path, file_name, data):
    """
    
    """
    with open(path+file_name, "w") as outfile:
        json.dump(data, outfile, indent=4, ensure_ascii=False)
        print(f"\n\nSuccessfully saved: {file_name}")

### all data

In [38]:
df = pd.read_csv("./data/All_cate.csv")
df.tail(40)

,name_of_cate,original_url
0,གདོང་ཤོག,http://www.tbwriters.com/
1,སྙན་ངག,https://www.tbwriters.com/?cat=4
2,ཟིན་བྲིས།,https://www.tbwriters.com/?cat=3
3,དཔྱད་རྩོམ།,https://www.tbwriters.com/?cat=2
4,ལྷུག་རྩོམ།,https://www.tbwriters.com/?cat=6
5,བསྒྱུར་རྩོམ།,https://www.tbwriters.com/?cat=5
6,སྒྲུང་རྩོམ།,https://www.tbwriters.com/?cat=7
7,ལོ་རྒྱུས།,https://www.tbwriters.com/?cat=8
8,འབོལ་རྩོམ།,https://www.tbwriters.com/?cat=9
9,བརྡ་ཁྱབ།,https://www.tbwriters.com/?cat=10


In [39]:
df.shape

(11, 2)

In [44]:
sample_df = df
sample_df.shape
sample_df

,name_of_cate,original_url
0,གདོང་ཤོག,http://www.tbwriters.com/
1,སྙན་ངག,https://www.tbwriters.com/?cat=4
2,ཟིན་བྲིས།,https://www.tbwriters.com/?cat=3
3,དཔྱད་རྩོམ།,https://www.tbwriters.com/?cat=2
4,ལྷུག་རྩོམ།,https://www.tbwriters.com/?cat=6
5,བསྒྱུར་རྩོམ།,https://www.tbwriters.com/?cat=5
6,སྒྲུང་རྩོམ།,https://www.tbwriters.com/?cat=7
7,ལོ་རྒྱུས།,https://www.tbwriters.com/?cat=8
8,འབོལ་རྩོམ།,https://www.tbwriters.com/?cat=9
9,བརྡ་ཁྱབ།,https://www.tbwriters.com/?cat=10


In [45]:
file_name_code = "tbwriters"

In [46]:
for index, row in sample_df.iterrows():
    # print(row)
    name_of_cate = row["name_of_cate"]
    original_url = row["original_url"]

    custom_url = original_url
    print(row)
    if custom_url == "http://www.tbwriters.com/":
        custom_url = "http://www.tbwriters.com"

    article_tag = name_of_cate
    key_code = "Page " + article_tag + " "
    all_links = loop_article_page(custom_url, key_code)
    
    print(f"Total page in {article_tag}: {len(all_links['Data'])}")
    check_error_in_links(all_links['Data'], key_code, print_each_error=True)

    # Saving the final file
    path = "./data/links/"
    file_name = f"{file_name_code}_ALL_link_{article_tag}.json"
    save_json(path, file_name, all_links['Data'])

name_of_cate                     གདོང་ཤོག
original_url    http://www.tbwriters.com/
Name: 0, dtype: object
http://www.tbwriters.com Page གདོང་ཤོག 


  0%|          | 1/999 [00:08<2:20:05,  8.42s/it]

http://www.tbwriters.com/?paged=1 True


  0%|          | 2/999 [00:11<1:30:30,  5.45s/it]

http://www.tbwriters.com/?paged=2 True


...

https://www.tbwriters.com/?cat=1&paged=5 True


  1%|          | 6/999 [00:35<1:40:45,  6.09s/it]

https://www.tbwriters.com/?cat=1&paged=6 True


  1%|          | 6/999 [00:40<1:50:47,  6.69s/it]

https://www.tbwriters.com/?cat=1&paged=7 False
Total page in ངེད་ཀྱི་སྐོར།: 7
Total error in Page ངེད་ཀྱི་སྐོར། : 0


Successfully saved: tbwriters_ALL_link_ངེད་ཀྱི་སྐོར།.json


In [47]:
import os

In [48]:
directory = './data/links/'
json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
json_files

['./data/links/tbwriters_ALL_link_སྙན་ངག.json',
 './data/links/tbwriters_ALL_link_ཟིན་བྲིས།.json',
 './data/links/tbwriters_ALL_link_གདོང་ཤོག.json',
 './data/links/tbwriters_ALL_link_དཔྱད་རྩོམ།.json',
 './data/links/tbwriters_ALL_link_ལྷུག་རྩོམ།.json',
 './data/links/tbwriters_ALL_link_བསྒྱུར་རྩོམ།.json',
 './data/links/tbwriters_ALL_link_སྒྲུང་རྩོམ།.json',
 './data/links/tbwriters_ALL_link_ལོ་རྒྱུས།.json',
 './data/links/tbwriters_ALL_link_འབོལ་རྩོམ།.json',
 './data/links/tbwriters_ALL_link_བརྡ་ཁྱབ།.json',
 './data/links/tbwriters_ALL_link_ངེད་ཀྱི་སྐོར།.json']

In [49]:
len(json_files)

11